# Demo for data_prep, hmm_training, classifier, and hmm_test

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

from CharacterFeatureExtractor import featureExtractor
from dataprep import *
from modeltrain import *
from classifier import *
from hmm_test import *
from createDataSet import *
from DrawCharacter import DrawCharacter
from PattRecClasses import HMM_TA
from hmm_gen import hmm_gen


## Training Model

In [2]:
### data prep
db_name = "database"
data_features = pd.read_pickle(r'data/' + db_name + '_features.cdb')
data_labels = pd.read_pickle(r'data/' + db_name + '_labels.cdb')
class_state_nums = np.array([2,5,5,5,5,5,5,6,5,5])

train_data, test_data, data_labels = dataprep(db_name, nr_test=5)
hm_learn, train_acc = modeltrain(train_data, data_labels, 12, class_state_nums, longest_sample = True, useprint=False)
accuracies, result_labels_list = hmm_test(hm_learn,test_data, data_labels)

    


Database read is  database
Labels used are  ['A', 'C', 'K', 'P', 'X', 'T', '+', 'N', 'V', '4']
Total training samples are  15  and testing samples are  5 


 ------------ CHARACTER  A ------------
Number of states:  2
Avg probability for entire sequence over test samples is -3.298671264230808  (log),  3.6932207958803365 %
Normalized score:  369.32207958803366

 ------------ CHARACTER  C ------------
Number of states:  5
Avg probability for entire sequence over test samples is -12.926138195285455  (log),  0.0002433601752572096 %
Normalized score:  24.33601752572096

 ------------ CHARACTER  K ------------
Number of states:  5
Avg probability for entire sequence over test samples is -16.372174309081682  (log),  7.75630789984598e-06 %
Normalized score:  0.775630789984598

 ------------ CHARACTER  P ------------
Number of states:  5
Avg probability for entire sequence over test samples is -13.892245889130054  (log),  9.261349402266299e-05 %
Normalized score:  9.2613494022663

 -----------

In [10]:
print("---------------- RESULTS: ---------------- ")
for k in range(len(data_labels)):
    print("\n Character: ", data_labels[k])
    print("Test accuracy: ",accuracies[k], "and resulting labels: ",result_labels_list[k])
    print("Training accuracy: (log)", train_acc[k])

---------------- RESULTS: ---------------- 

 Character:  A
Test accuracy:  0.6 and resulting labels:  ['A', 'P', 'P', 'A', 'A']
Training accuracy: (log) -3.298671264230808

 Character:  C
Test accuracy:  1.0 and resulting labels:  ['C', 'C', 'C', 'C', 'C']
Training accuracy: (log) -12.926138195285455

 Character:  K
Test accuracy:  0.4 and resulting labels:  ['K', 'K', 'A', 'A', 'A']
Training accuracy: (log) -16.372174309081682

 Character:  P
Test accuracy:  0.2 and resulting labels:  ['A', 'A', 'A', 'A', 'P']
Training accuracy: (log) -13.892245889130054

 Character:  X
Test accuracy:  0.8 and resulting labels:  ['X', '4', 'X', 'X', 'X']
Training accuracy: (log) -11.208775982648183

 Character:  T
Test accuracy:  0.4 and resulting labels:  ['4', 'T', 'A', 'P', 'T']
Training accuracy: (log) -13.853658775153216

 Character:  +
Test accuracy:  0.2 and resulting labels:  ['A', 'A', '+', 'A', 'A']
Training accuracy: (log) -12.185319817315312

 Character:  N
Test accuracy:  0.0 and resulti

In [11]:
# If you want to save!

import os
import pickle
import pandas as pd

name = input("What do you want to save the files as?")
    
with open(name, "wb") as fp: # Pickling
    pickle.dump(hm_learn, fp)

What do you want to save the files as?hmm_demo


In [4]:
# ### data prep
# db_name = "database_inc_sampchar"
# data_features = pd.read_pickle(r'data/' + db_name + '_features.cdb')
# data_labels = pd.read_pickle(r'data/' + db_name + '_labels.cdb')

# num_state_list = np.array([2, 3, 4, 5, 6, 7])
# train_acc_list = []
# big_result_labels_list = []
# test_acc_list = []
# for i in num_state_list:
#     print("++++++++++++++++++++++TRAINING OF HMM MODELS WITH " + str(i) + " STATES++++++++++++++++++++++")
#     train_data, test_data, data_labels = dataprep(db_name, nr_test=5)
#     hm_learn, train_acc = modeltrain(train_data, data_labels, 20, i, longest_sample= True, useprint=False)
#     accuracies, result_labels_list = hmm_test(hm_learn,test_data, data_labels)
#     train_acc_list.append(train_acc)
#     test_acc_list.append(accuracies)
#     big_result_labels_list.append(result_labels_list)
    


# #print(result_labels_list)

In [5]:
db_name = "database_inc_sampchar"
data_features = pd.read_pickle(r'data/' + db_name + '_features.cdb')
data_labels = pd.read_pickle(r'data/' + db_name + '_labels.cdb')
print(data_labels)
print(len(data_features))
print(train_acc)


['A', 'C', 'K', 'P', 'X', 'T', '+', 'N', 'V', '4']
10
[-3.298671264230808, -12.926138195285455, -16.372174309081682, -13.892245889130054, -11.208775982648183, -13.853658775153216, -12.185319817315312, -22.788263216350106, -10.01827267638159, -12.429617378730805]
